In [1]:
## 3482 spectrogram images should be trained on cnn 

In [2]:
import keras
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

In [6]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob 
import librosa as lr

In [8]:
data_dir ='C:/Users/salta/Desktop/LAPTOP-HHJUF3BH/Asus/Saltanat/Documents/COVID19/split_sound'
audio_files =glob(data_dir + '/*.wav')
len(audio_files)

3482

In [9]:
val=[]
myDict = {}
for file in range(0, len(audio_files), 1):
    val = audio_files[file]
    myDict[file] = val
df = pd.DataFrame.from_dict(myDict, orient='index')

In [14]:
new = df[0].str.split('split_sound', n = 2, expand = True) 
  
# making separate first name column from new data frame 
#df["address"]= new[0] 
  
# making separate last name column from new data frame 
#df[0]= new[1] 

In [32]:
#new

In [18]:
#df[0].replace('.wav', '',regex=True,inplace=True)

In [24]:
new[1] = new[1].str.replace('\\', '',regex = True)
new[1] = new[1].str.replace('.wav', '',regex = True)

In [33]:
#new

In [26]:
new[['filename', 'ext']] =new[1].str.split("_", n = 2, expand = True) 

In [34]:
#new

In [28]:
new = new.drop(columns=[0, 1,'ext'])

In [35]:
#new

In [30]:
new['image_filename'] = new.index 

In [36]:
new.to_csv(r'C:/Users/salta/Documents/ra/image_filenames_sound_filenames.csv', index = False)

In [57]:
im_id = pd.read_csv('C:/Users/salta/Documents/ra/image_filenames_sound_filenames.csv') 

In [47]:
json_corr =pd.read_json(r'C:/Users/salta/Documents/ra/records_1625681524729.json')

In [51]:
corr_val = json_corr[['correlation','fileName', 'coughType', 'cough', 'smoking', 'gravity']]

In [55]:
#corr_val
#corr_val = json_corr[['correlation','fileName']]
corr_val['fileName']=corr_val['fileName'].str.replace(".aac", "")

In [66]:
#im_id.dtypes
#corr_val.dtypes
im_id['filename'] = im_id['filename'].apply(str)

In [67]:
join_im_id_correlation = corr_val.join(im_id.set_index('filename'), on='fileName')

In [69]:
join_im_id_correlation.dropna(subset = ["image_filename"], inplace=True)

In [71]:
join_im_id_correlation.to_csv(r'C:/Users/salta/Documents/ra/3482im_filenames.csv', index = False)

In [73]:
join_im_id_correlation.drop(join_im_id_correlation.loc[join_im_id_correlation['correlation']=='Other'].index, inplace=True)

In [76]:
join_im_id_correlation.drop(join_im_id_correlation.loc[join_im_id_correlation['correlation']=='Tuberculosis'].index, inplace=True)

In [77]:
join_im_id_correlation.drop(join_im_id_correlation.loc[join_im_id_correlation['correlation']=='Cardio'].index, inplace=True)

In [78]:
join_im_id_correlation.drop(join_im_id_correlation.loc[join_im_id_correlation['correlation']=='Cancer'].index, inplace=True)

In [79]:
join_im_id_correlation['correlation'].value_counts()

COVID-19                497
Allergies               497
Asthma                  322
Pneumonia/Bronchitis    150
Name: correlation, dtype: int64

In [83]:
join_im_id_correlation.dtypes

correlation       object
fileName          object
coughType         object
cough             object
smoking           object
gravity           object
image_filename     int64
dtype: object

In [82]:
join_im_id_correlation['image_filename'] = join_im_id_correlation['image_filename'].apply(np.int64)

In [ ]:
#traindf=pd.read_csv("C:/Users/salta/Desktop/LAPTOP-HHJUF3BH/Asus/Saltanat/Documents/COVID19/trainim.csv",dtype=str)
#testdf=pd.read_csv("C:/Users/salta/Desktop/LAPTOP-HHJUF3BH/Asus/Saltanat/Documents/COVID19/testim.csv",dtype=str)

#traindf["filename"]=traindf["filename"].apply(append_ext)
#testdf["filename"]=testdf["filename"].apply(append_ext)
#traindf = traindf.drop(columns=['Unnamed: 0'])
#testdf = testdf.drop(columns=['Unnamed: 0'])

In [86]:
#C:\Users\salta\Documents\ra\spectrograms\train

directory ='C:/Users/salta/Documents/ra/spectrograms/train'
im_files =glob(directory + '/*.png')
len(im_files)

2786

In [113]:

directory2 ='C:/Users/salta/Documents/ra/spectrograms/test'
im_files_test =glob(directory2 + '/*.png')
len(im_files_test)

696

In [87]:
val1=[]
myDict1 = {}
for file in range(0, len(im_files), 1):
    val = im_files[file]
    myDict1[file] = val
im_f = pd.DataFrame.from_dict(myDict1, orient='index')

In [114]:
val2=[]
myDict2 = {}
for file in range(0, len(im_files_test), 1):
    val2 = im_files_test[file]
    myDict2[file] = val2
im_f_test = pd.DataFrame.from_dict(myDict2, orient='index')

In [90]:
im_f = im_f[0].str.split('train', n = 2, expand = True) 

In [120]:
im_f_test = im_f_test[0].str.split('test', n = 2, expand = True) 

In [95]:
im_f = im_f.drop(columns=[0])

In [123]:
im_f_test = im_f_test.drop(columns=[0])

In [99]:
def append_ext(fn):
    return fn+".png"

In [103]:
join_im_id_correlation['image_filename'] = join_im_id_correlation['image_filename'].astype('str')

In [104]:
join_im_id_correlation['image_filename']=join_im_id_correlation['image_filename'].apply(append_ext)

In [106]:
im_f = im_f.rename(columns={1: 'name'})

In [125]:
im_f_test = im_f_test.rename(columns={1: 'name'})

In [138]:
train_im_chunks = im_f.join(join_im_id_correlation.set_index('image_filename'), on= 'name')
train_im_chunks  = train_im_chunks.dropna(subset=['correlation'])

In [141]:
train_im_chunks.to_csv(r'C:/Users/salta/Documents/ra/train.csv', index =False)
#join_im_id_correlation.to_csv(r'C:/Users/salta/Documents/ra/3482im_filenames.csv', index = False)

In [111]:
#train_im_chunks['image_filename'].value_counts()
train_im_chunks['image_filename'].isnull().values.sum()

0

In [132]:
test_im_chunks = im_f_test.join(join_im_id_correlation.set_index('image_filename'), on= 'name')

In [135]:
test_im_chunks  = test_im_chunks.dropna(subset=['correlation'])

In [137]:
## 1 second chunks spectrogram images were saved in test and train folders, 
## we have 223 rows, which is 223 images and their features will be extracted with cnn, 
## because we dropped other, cardio, tuberculosis, no diagnosis classes
test_im_chunks.to_csv(r'C:/Users/salta/Documents/ra/test.csv', index =False)

## CNN 

In [142]:
traindf=pd.read_csv("C:/Users/salta/Documents/ra/train.csv",dtype=str)
testdf=pd.read_csv("C:/Users/salta/Documents/ra/test.csv",dtype=str)

#traindf["filename"]=traindf["filename"].apply(append_ext)
#testdf["filename"]=testdf["filename"].apply(append_ext)
#traindf = traindf.drop(columns=['Unnamed: 0'])
#testdf = testdf.drop(columns=['Unnamed: 0'])

In [161]:
#datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)
train_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="C:/Users/salta/Documents/ra/spectrograms/train/",
x_col="name",
y_col="correlation",
#subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="C:/Users/salta/Documents/ra/spectrograms/test/",
x_col="name",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 1243 validated image filenames belonging to 4 classes.
Found 223 validated image filenames.


valid_generator=datagen.flow_from_dataframe(
dataframe=traindf,
directory="C:/Users/salta/Documents/ra/spectrograms/train/",
x_col="name",
y_col="correlation",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))


In [162]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [163]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [164]:
model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=10)

Epoch 1/10
38/38 [==============================] - 13s 268ms/step - loss: 1.3033 - accuracy: 0.3505 - val_loss: 1.3281 - val_accuracy: 0.3058
Epoch 2/10
38/38 [==============================] - 9s 229ms/step - loss: 1.2754 - accuracy: 0.3490 - val_loss: 1.3333 - val_accuracy: 0.3165
Epoch 3/10
38/38 [==============================] - 9s 238ms/step - loss: 1.2823 - accuracy: 0.3583 - val_loss: 1.3146 - val_accuracy: 0.3345
Epoch 4/10
38/38 [==============================] - 9s 232ms/step - loss: 1.2822 - accuracy: 0.3492 - val_loss: 1.3037 - val_accuracy: 0.3453
Epoch 5/10
38/38 [==============================] - 9s 242ms/step - loss: 1.2819 - accuracy: 0.3259 - val_loss: 1.3380 - val_accuracy: 0.2482
Epoch 6/10
38/38 [==============================] - 9s 232ms/step - loss: 1.2545 - accuracy: 0.3721 - val_loss: 1.3038 - val_accuracy: 0.2842
Epoch 7/10
38/38 [==============================] - 9s 232ms/step - loss: 1.2912 - accuracy: 0.3188 - val_loss: 1.3172 - val_accuracy: 0.3309
Epoch

In [169]:
model.evaluate(valid_generator,
steps=STEP_SIZE_TEST)

6/6 [==============================] - 1s 150ms/step - loss: 1.3057 - accuracy: 0.3352


[1.3057090044021606, 0.3351648449897766]

In [165]:
traindf['correlation'].value_counts()

Allergies               430
COVID-19                413
Asthma                  307
Pneumonia/Bronchitis     93
Name: correlation, dtype: int64

In [166]:
testdf['correlation'].value_counts()

COVID-19                84
Allergies               67
Pneumonia/Bronchitis    57
Asthma                  15
Name: correlation, dtype: int64

In [170]:
test_generator.reset()
pred=model.predict(test_generator,
steps=len(test_generator),
verbose=1)

7/7 [==============================] - 2s 154ms/step


In [171]:
predicted_class_indices=np.argmax(pred,axis=1)

In [172]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [173]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [174]:
results['Predictions'].value_counts()

Allergies    137
COVID-19      86
Name: Predictions, dtype: int64

In [175]:
from sklearn.metrics import classification_report

In [176]:
y_true = testdf['correlation']
y_pred = results['Predictions']

In [177]:
print(classification_report(y_true, y_pred, labels=['COVID-19','Allergies','Pneumonia/Bronchitis','Asthma']))

                      precision    recall  f1-score   support

            COVID-19       0.53      0.55      0.54        84
           Allergies       0.38      0.78      0.51        67
Pneumonia/Bronchitis       0.00      0.00      0.00        57
              Asthma       0.00      0.00      0.00        15

            accuracy                           0.44       223
           macro avg       0.23      0.33      0.26       223
        weighted avg       0.32      0.44      0.36       223



C:\Users\salta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\salta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\salta\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
